# Contenido de Refuerzo – CRUD de Tareas con FastAPI + PostgreSQL

## Objetivo General

Afianzar los conocimientos adquiridos sobre FastAPI, SQLAlchemy y PostgreSQL mediante teoría aplicada y ejercicios prácticos progresivos que fortalezcan la comprensión del desarrollo de APIs RESTful profesionales con persistencia de datos.

---


## PARTE 1: REVISIÓN TEÓRICA

### 1. ¿Qué es una API REST?

* Arquitectura basada en recursos, identificados por URLs.
* Se usa HTTP con métodos (`GET`, `POST`, `PUT`, `DELETE`, etc).
* Las operaciones deben ser **idempotentes** (salvo `POST`).

### 2. ¿Qué es FastAPI?

* Framework web moderno y rápido.
* Basado en `Starlette` y `Pydantic`.
* Ideal para construir APIs con validaciones automáticas y documentación interactiva.

### 3. ¿Qué es SQLAlchemy?

* ORM de Python.
* Permite trabajar con bases de datos relacionales como si fueran objetos de Python.
* Incluye modelos declarativos, sesiones y expresiones SQL seguras.

### 4. ¿Qué es PostgreSQL?

* Sistema de base de datos relacional potente y open source.
* Soporta integridad referencial, transacciones, concurrencia y extensiones avanzadas.

---


## PARTE 2: EJERCICIOS PRÁCTICOS PROGRESIVOS

### Nivel 1 – Fundamentos

#### Ejercicio 1: Crear tu primer endpoint

**Objetivo:** Crear un endpoint `GET /ping` que devuelva `"pong"`.




In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/ping")
def ping():
    return {"mensaje": "pong"}


 *Validar que FastAPI responde y Swagger funciona.*

---


#### Ejercicio 2: Definir tu primer modelo de datos con Pydantic

**Objetivo:** Crear el modelo `TareaBase` con los campos `titulo`, `descripcion`, `completado`.



In [ ]:
from pydantic import BaseModel

class TareaBase(BaseModel):
    titulo: str
    descripcion: str
    completado: bool


*Usar este modelo para validar los datos de entrada en `POST`.*

---


### Nivel 2 – Conexión con la base de datos

#### Ejercicio 3: Crear el modelo ORM

**Objetivo:** Usa SQLAlchemy para crear una tabla llamada tareas con los campos: id (int, PK), titulo (str), descripcion (str), completado (bool).


In [ ]:
from sqlalchemy import Column, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class Tarea(Base):
    __tablename__ = "tareas"

    id = Column(Integer, primary_key=True, index=True)
    titulo = Column(String, nullable=False)
    descripcion = Column(String)
    completado = Column(Boolean, default=False)


*Revisar que se cree correctamente en la base de datos.*

---


#### Ejercicio 4: Crear una tarea y guardarla en la base de datos

**Objetivo:** Implementa el endpoint POST /tareas que reciba una TareaBase, cree una nueva tarea en la base de datos y la devuelva


In [ ]:
from fastapi import Depends
from sqlalchemy.orm import Session

from .database import get_db

@app.post("/tareas")
def crear_tarea(tarea: TareaBase, db: Session = Depends(get_db)):
    nueva_tarea = Tarea(**tarea.dict())
    db.add(nueva_tarea)
    db.commit()
    db.refresh(nueva_tarea)
    return nueva_tarea


---

### Nivel 3 – Lectura, actualización y eliminación

#### Ejercicio 5: Obtener todas las tareas

**Objetivo:** Crea el endpoint GET /tareas para devolver todas las tareas almacenadas.


In [ ]:
@app.get("/tareas")
def obtener_tareas(db: Session = Depends(get_db)):
    return db.query(Tarea).all()



#### Ejercicio 6: Buscar una tarea por ID

**Objetivo:** Crea GET /tareas/{id} para devolver una tarea específica por ID. Si no existe, devuelve 404


In [ ]:
from fastapi import HTTPException

@app.get("/tareas/{id}")
def obtener_tarea(id: int, db: Session = Depends(get_db)):
    tarea = db.query(Tarea).filter(Tarea.id == id).first()
    if tarea is None:
        raise HTTPException(status_code=404, detail="Tarea no encontrada")
    return tarea



#### Ejercicio 7: Actualizar una tarea

**Objetivo:** Crea PUT /tareas/{id} para actualizar una tarea existente


In [ ]:
@app.put("/tareas/{id}")
def actualizar_tarea(id: int, datos: TareaBase, db: Session = Depends(get_db)):
    tarea = db.query(Tarea).filter(Tarea.id == id).first()
    if tarea is None:
        raise HTTPException(status_code=404, detail="Tarea no encontrada")
    for attr, value in datos.dict().items():
        setattr(tarea, attr, value)
    db.commit()
    db.refresh(tarea)
    return tarea


#### Ejercicio 8: Eliminar una tarea

**Objetivo:** Crea DELETE /tareas/{id} para eliminar una tarea existente


In [ ]:
@app.delete("/tareas/{id}")
def eliminar_tarea(id: int, db: Session = Depends(get_db)):
    tarea = db.query(Tarea).filter(Tarea.id == id).first()
    if tarea is None:
        raise HTTPException(status_code=404, detail="Tarea no encontrada")
    db.delete(tarea)
    db.commit()
    return {"mensaje": "Tarea eliminada"}

